# 1- Alloy2CD: from XML instance

In [53]:
#XML library
import xml.etree.ElementTree as ET

In [77]:
#get the root element
root = ET.parse('generatedInstance-1.xml').getroot()

In [78]:
# get last class diagram
cds = [e for e in root.findall('instance/sig') if e.attrib['label']=='this/CD'][0]
lastCD = cds.getchildren()[-1].attrib['label']

c:\users\oussama\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  


In [79]:
# get the list of classes
classes = [c for c in root.findall('instance/field') if c.attrib['label']=='classes'][0].getchildren()[:-1]
classNames = [c.getchildren()[1].attrib['label'] for c in classes
             if c.getchildren()[0].attrib['label']==lastCD]

c:\users\oussama\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  """Entry point for launching an IPython kernel.
c:\users\oussama\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [85]:
# get the list of fields
fieldsElem = [c for c in root.findall('instance/field') if c.attrib['label']=='features'][0].getchildren()[:-1]
fields = {}
for fe in fieldsElem:
    ch = fe.getchildren()
    if ch[0].attrib['label'] == lastCD:
        c = ch[1].attrib['label'] 
        f = ch[2].attrib['label'] 
        t = ch[3].attrib['label']
        if c in fields.keys():
            fields[c].append({
                'fieldName': f,
                'fieldType': t.split('type_')[1]
            })
        else:
            fields[c] = [{
                'fieldName': f,
                'fieldType': t.split('type_')[1]
            }]
            

c:\users\oussama\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  
c:\users\oussama\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  """


In [95]:
# get the module name
module = [e for e in root.findall('source')][0].attrib['content'].split('module')[1].split('\n')[0].strip()

In [120]:
# output: alloy2cd
classesStr = ''
for c in classNames:
    line = '\tclass {className}'.format(className=c.split('$')[0])
    if not(c in fields.keys()):
        classesStr+=(line+';\n')
        continue
    fieldsTxt = ''
    for f in fields[c]:
        fieldsTxt+= f'{f["fieldType"].split("$")[0]} {f["fieldName"].split("$")[0]};'
        
    line+= f'{{{fieldsTxt}}}'

    classesStr+= (line+'\n')

In [121]:
classesStr

'\tclass Unit{int health;}\n\tclass Tank;\n\tclass Soldier;\n'

In [130]:
output = f"""
package alloy2cd.refactoring;

classdiagram {module} {{0;

{classesStr}
  
}}

"""

In [131]:
print(output)


package alloy2cd.refactoring;

classdiagram umlp2alloy/CD2Alloy4CDModule {

	class Unit{int health;}
	class Tank;
	class Soldier;

  
}




In [137]:
# Write output to file
path = r'D:\Eclipse\eclipse-epsilon-1.5-win32-x86_64\eclipse\workspace\SoftwareAnalysisProject\resources\Alloy2CdRefactored.cd'
file = open(path, "w")
file.write(output)
file.close()

# 2- Alloy2CD: from alloy source code

In [363]:
import re
import pandas as pd

In [364]:
filename = "example-1.als"
lines = ""

In [365]:
with open(filename, "r") as f:
    lines = f.read()
print(lines)  

module alloy2cd

open util/boolean


abstract sig Vehicle   {
}

sig Car extends Vehicle {
	wheels: some Wheel,
	isdeflated: Bool
}

one sig Person {
	name: String,
	age: Int, 
	cars: some Car
}


sig Wheel  {
	cars: one Car
}

pred exp {
	 all p: Person | p.name in ("John" + "Paul" + "Michael")
} run exp for 3



In [366]:
# find package
module = re.findall('\s*module\s+(\w+)', lines)[0]

In [367]:
module

'alloy2cd'

In [368]:
# find sigs
sigs = re.findall('(\w*)(\w*)\s*sig\s+(\w+)\s+(?:extends)?\s*(\w+)?\s*\{([^{]+)\}', lines)

In [369]:
sigs

[('abstract', '', 'Vehicle', '', '\n'),
 ('', '', 'Car', 'Vehicle', '\n\twheels: some Wheel,\n\tisdeflated: Bool\n'),
 ('one',
  '',
  'Person',
  '',
  '\n\tname: String,\n\tage: Int, \n\tcars: some Car\n'),
 ('', '', 'Wheel', '', '\n\tcars: one Car\n')]

In [370]:
pd.DataFrame(sigs)

,0,1,2,3,4
0,abstract,,Vehicle,,\n
1,,,Car,Vehicle,"\n\twheels: some Wheel,\n\tisdeflated: Bool\n"
2,one,,Person,,"\n\tname: String,\n\tage: Int, \n\tcars: some ..."
3,,,Wheel,,\n\tcars: one Car\n


In [371]:
classNames = [s[2] for s in sigs]
classNames

['Vehicle', 'Car', 'Person', 'Wheel']

In [372]:
classes = []
for s in sigs:
    features = re.findall('(\w*)\s*:\s*(one|some|lone)?\s*(\w*)', s[4])
    c = {
        'name': s[2], 
        'isAbstract': 'abstract' in s[0:2],
        'extends': s[3],
        'features': [],
        'associations': []
     }
    for f in features:
        if(f[2] in classNames):
            c['associations'].append({
            'name': f[0],
            'arity': f[1],
            'target': f[2] 
            })
        else:
            c['features'].append({
                'name': f[0],
                'arity': f[1],
                'type': f[2] 
            })
    classes.append(c)

In [373]:
classes

[{'name': 'Vehicle',
  'isAbstract': True,
  'extends': '',
  'features': [],
  'associations': []},
 {'name': 'Car',
  'isAbstract': False,
  'extends': 'Vehicle',
  'features': [{'name': 'isdeflated', 'arity': '', 'type': 'Bool'}],
  'associations': [{'name': 'wheels', 'arity': 'some', 'target': 'Wheel'}]},
 {'name': 'Person',
  'isAbstract': False,
  'extends': '',
  'features': [{'name': 'name', 'arity': '', 'type': 'String'},
   {'name': 'age', 'arity': '', 'type': 'Int'}],
  'associations': [{'name': 'cars', 'arity': 'some', 'target': 'Car'}]},
 {'name': 'Wheel',
  'isAbstract': False,
  'extends': '',
  'features': [],
  'associations': [{'name': 'cars', 'arity': 'one', 'target': 'Car'}]}]

In [374]:
display(pd.DataFrame(classes))
for c in classes:
    display(pd.DataFrame(c['features']))
    display(pd.DataFrame(c['associations']))

,name,isAbstract,extends,features,associations
0,Vehicle,True,,[],[]
1,Car,False,Vehicle,"[{'name': 'isdeflated', 'arity': '', 'type': '...","[{'name': 'wheels', 'arity': 'some', 'target':..."
2,Person,False,,"[{'name': 'name', 'arity': '', 'type': 'String...","[{'name': 'cars', 'arity': 'some', 'target': '..."
3,Wheel,False,,[],"[{'name': 'cars', 'arity': 'one', 'target': 'C..."


""


""


,name,arity,type
0,isdeflated,,Bool


,name,arity,target
0,wheels,some,Wheel


,name,arity,type
0,name,,String
1,age,,Int


,name,arity,target
0,cars,some,Car


""


,name,arity,target
0,cars,one,Car


In [375]:
# cd output: classes + attributes
classesStr = ''

for c in classes:
    line = '\t{abstract}class {className}{extends}'.format(abstract='abstract ' if c['isAbstract'] else '', 
                                                           className=c['name'], 
                                                           extends = '' if (c['extends']=='') else (' extends '+c['extends']))
#     if not(c in fields.keys()):
#         classesStr+=(line+';\n')
#         continue
    fieldsTxt = ''
    for f in c['features']:
        fieldsTxt+= f' {f["type"]} {f["name"]}; '
        
    line+= f' {{{fieldsTxt}}}'

    classesStr+= (line+'\n')

In [376]:
print(classesStr)

	abstract class Vehicle {}
	class Car extends Vehicle { Bool isdeflated; }
	class Person { String name;  Int age; }
	class Wheel {}



In [377]:
# associations
associations = []

for c in classes:
    for a in c['associations']:
        a['source'] = c['name']
        a['arity2'] = '-'
        b = False
        for rel in associations:
            if rel['source']==a['target'] and a['source']==rel['target']:
                rel['type'] = '--'
                rel['arity2'] = a['arity']
                b = True
                break
        if not b:
            a['type'] = '->'
            associations.append(a)
associations

[{'name': 'wheels',
  'arity': 'some',
  'target': 'Wheel',
  'source': 'Car',
  'arity2': 'one',
  'type': '--'},
 {'name': 'cars',
  'arity': 'some',
  'target': 'Car',
  'source': 'Person',
  'arity2': '-',
  'type': '->'}]

In [378]:
associationsStr = ''

def get_arity(argument):
    switcher = {
        '': '[0..*]',
        'some': '[1..*]',
        'one': '[1..1]',
        'lone': '[0..1]'
    }
    return switcher.get(argument, '')

for a in associations:
#     association WorkPlace [1] Employee (of) -> (worksIn) Address [1..3];
    name = a['name']
    arity = get_arity(a['arity'])
    arity2 = get_arity(a['arity2'])
    src = a['source']
    tgt = a['target']
    atype = a['type']
    associationsStr += f'\tassociation {name} {src} {arity2} {atype} {tgt} {arity};\n'

In [379]:
print(associationsStr)

	association wheels Car [1..1] -- Wheel [1..*];
	association cars Person  -> Car [1..*];



In [380]:
output = f"""
package alloy2cd;

classdiagram {module} {{

{classesStr}

{associationsStr}
  
}}

"""

In [381]:
# print(lines)
# print('************************************************************')
print(output)


package alloy2cd;

classdiagram alloy2cd {

	abstract class Vehicle {}
	class Car extends Vehicle { Bool isdeflated; }
	class Person { String name;  Int age; }
	class Wheel {}


	association wheels Car [1..1] -- Wheel [1..*];
	association cars Person  -> Car [1..*];

  
}


